In [1]:
import sys
import time

from src.efficentnet_train import data_loader,data_preprocessing
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from src.efficentfacenet import face_descriptor
from torchvision import transforms



In [4]:

dataset_path="../dataset/preprocessed/lfw_prep"

In [3]:
imgs_dict=data_loader.get_imgs_dict(dataset_path)

 100.0 % of the folders processed img dict loaded in 0.48 m


In [4]:
len(imgs_dict)

12646

In [5]:
train_data_loader=data_loader.data_generator(32,imgs_dict,dataset_path)

In [6]:
a,p,n=next(train_data_loader)

In [7]:
a.shape

(32, 160, 160, 3)

In [8]:
# for i in range(30):
#     _,ax=plt.subplots(1,3)
#     ax[0].set_title("Anchor")
#     ax[0].imshow(a[i])
#
#     ax[1].set_title("Positive")
#     ax[1].imshow(p[i])
#
#     ax[2].set_title("Negative")
#     ax[2].imshow(n[i])

In [5]:
model=face_descriptor.FaceDescriptorModel(download_weights=True,version="efficientnet_b1")
trans=transforms.Compose([transforms.ToTensor(),transforms.Resize((240,240))])

In [6]:
dataset=data_loader.FacesDataset(dataset_path,1024,trans)

In [8]:
from torch import optim
from torch.nn import TripletMarginLoss
import time
epochs=5
batch_size=16
lr=0.01
optimizer=optim.Adam(model.parameters(),lr=lr)
loss_function=TripletMarginLoss()
train_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)
data_size=float(len(train_loader.dataset))

In [14]:
print()

In [9]:
for p in model.parameters():
    p.requires_grad=True

In [10]:
for e in range(epochs):
    loss_sum=0.0
    ts=time.time()
    for anchor_img,positive_img,negative_img in train_loader:
        optimizer.zero_grad()
        anchor_img.requires_grad=True
        positive_img.requires_grad=True
        negative_img.requires_grad=True

        anchor_vector=model(anchor_img)
        positive_vector=model(positive_img)
        negative_vector=model(negative_img)

        loss=loss_function(anchor_img,positive_img,negative_img)
        loss.backward()

        loss_sum+=loss.item()*batch_size
        optimizer.step()
    te=time.time()
    sys.stdout.flush()
    sys.stdout.write("\r loss="+str(loss_sum/data_size)+" time for one epoch= "+str((te-ts)/60.0)[0:4])

 loss=0.8236878188326955 time for one epoch= 13.6

KeyboardInterrupt: 

In [ ]:
print()

In [14]:
model.classifier.requires_grad_()

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=128, bias=True)
)